# Library

In [48]:
!pip install PySastrawi

                                              0.0/210.6 kB ? eta -:--:--
     -                                        10.2/210.6 kB ? eta -:--:--
     -------                               41.0/210.6 kB 487.6 kB/s eta 0:00:01
     -------                               41.0/210.6 kB 487.6 kB/s eta 0:00:01
     -------                               41.0/210.6 kB 487.6 kB/s eta 0:00:01
     -----------------------------        174.1/210.6 kB 748.1 kB/s eta 0:00:01
     ------------------------------------ 210.6/210.6 kB 753.3 kB/s eta 0:00:00


In [49]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk import word_tokenize,sent_tokenize, FreqDist
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.model_selection import train_test_split

In [50]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\livia.LAPTOP-
[nltk_data]     KSNKPOSP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\livia.LAPTOP-
[nltk_data]     KSNKPOSP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load Data

In [73]:
df = pd.read_csv('D:/SEMESTER 8/TUGAS AKHIR/Bismillah TA/PER DATA AN DUNIAWI/Yang dicobain aja/Dataset_baru_BBM_multiclaass_preprocessed_labelling.csv')
df
# df.info()

,Unnamed: 0,tweet,Labels
0,0,@username sirait. sirait. kok otak kamu cetek ...,2
1,1,lain halnya islam yang mengelola sda minyak da...,1
2,2,kenaikan bahan bakar minyak seringkali diikuti...,0
3,3,"harga bahan bakar minyak naik, pemerintah dide...",0
4,4,"selesai zuhur angin bertiup kencang, entah apa...",0
...,...,...,...
9746,9746,adanya lembaga penyalur tersebut akan kian mem...,1
9747,9747,"harga bahan bakar minyak tidak naik, cuma peny...",2
9748,9748,sependapat dengan presiden jokowi bahwa sudah ...,1
9749,9749,ayo simak video di bawah ini video memperlihat...,0


In [74]:
if (df['Labels'].isna().sum() != 0):
  df.dropna(subset= ["Labels"],inplace = True)
else:
    pass
df['Labels'].isna().sum()

0

In [75]:
#check duplicate

df.duplicated(subset=['tweet']).sum()

3

In [76]:
df.drop_duplicates(subset=['tweet'],inplace=True)
df.reset_index(inplace=True)
len(df)

9748

In [77]:
X = df['tweet']
y = df['Labels']

In [78]:
X

0       @username sirait. sirait. kok otak kamu cetek ...
1       lain halnya islam yang mengelola sda minyak da...
2       kenaikan bahan bakar minyak seringkali diikuti...
3       harga bahan bakar minyak naik, pemerintah dide...
4       selesai zuhur angin bertiup kencang, entah apa...
                              ...                        
9743    adanya lembaga penyalur tersebut akan kian mem...
9744    harga bahan bakar minyak tidak naik, cuma peny...
9745    sependapat dengan presiden jokowi bahwa sudah ...
9746    ayo simak video di bawah ini video memperlihat...
9747    masih terpikirkan sama bapak gojek yang tadi, ...
Name: tweet, Length: 9748, dtype: object

In [79]:
y

0       2
1       1
2       0
3       0
4       0
       ..
9743    1
9744    2
9745    1
9746    0
9747    0
Name: Labels, Length: 9748, dtype: int64

In [80]:
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{0: 3120, 1: 3206, 2: 3422}

# Functions

In [81]:
ranges = [
    (0x1F600, 0x1F64F),  # Emoticons
    (0x1F910, 0x1F96B),  # Supplemental emoticons
    (0x1F300, 0x1F5FF),  # Miscellaneous symbols and pictographs
    (0x1F680, 0x1F6FF),  # Transport and map symbols
    (0x2600, 0x26FF),    # Miscellaneous symbols
    (0x2700, 0x27BF),    # Dingbats
    (0x1F1E0, 0x1F1FF),  # Flags (iOS)
    (0x1F9E7, 0x1F9FF),  # Supplemental flags
    (0x1F018, 0x1F9CF),  # Various symbols (e.g. skull, heart, star, etc.)
]

pattern = '[' + ''.join([chr(i) for r in ranges for i in range(r[0], r[1]+1)]) + ']'

def cleaning(text):
    #remove emoji
    text = re.sub(pattern, ' ', text)
    #remove url
    text = re.sub(r"http\S+", "", text)
    #remove hex characters
    text = re.sub(r'(\\x(.){2})', '', text)
    #remove newlines
    text = re.sub(r'\s+|\\n', ' ', text)
    #remove Retweet (RT) in a tweet
    text = re.sub(r'RT ','', text)
    #remove mention
    text = re.sub(r"@[A-Za-z0-9]+","", text)
    #remove hashtags
    text = re.sub(r'\B#\w*[a-zA-Z]+\w*','', text)
    #remove single char
    text = re.sub(r'\b[a-zA-Z]\b','', text)
    #remove number
    text = re.sub(r'\d+','', text)
    #remove punctuation / tanda baca
    text = re.sub(r'[^\w\s]+', ' ', text)
    #remove spaces at the beginning of tweet
    text = re.sub(r"/^\\s+/m", '', text)
    #remove multiplechar
    # Menghapus duplikasi karakter kecuali kata 'BBM' (baik huruf besar maupun kecil)
    text = re.sub(r'(?i)([^B]|B(?!BM)|B{2,})(?=[^BM\b])', r'\1', text)
    # Menghapus Tag HTML
    text = re.sub('<.*?>', ' ', text)
    return text

In [82]:
data_kbbi = [kamus.strip('\n') for kamus in open('kamus_gab - kamus_gab.tsv',encoding='utf-8')]
def change_word(text):
    kamus = {}
    for i in data_kbbi:
        (key,val) = i.split('\t')
        kamus[str(key)]=val

    final_string = ' '.join(str(kamus.get(word, word)) for word in text).split()
    return final_string

In [83]:
df_meaningless = pd.read_csv('meaningless.csv',header = None)
df_meaningless.head()

,0
0,kamu
1,aduh
2,ah
3,ahelah
4,amp


In [84]:
#dari nltk
nltk_stopword = stopwords.words('indonesian')
#dari sastrawi
stopword_id_factory  = StopWordRemoverFactory()
sastrawi_stopword = stopword_id_factory.get_stop_words()
meaningless_list = df_meaningless[0].tolist()

listStopword = list(set(nltk_stopword + sastrawi_stopword + meaningless_list))
exception_word = ['naik']
listStopword.remove(exception_word[0])
def stopwords_removal(words):
    return [word for word in words if word not in listStopword]

In [85]:
#mengembalikan token menjadi string tunggal kembali
def untokenize(text):
    text = ' '.join([t for t in text])
    return text

In [86]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
    text = stemmer.stem(text)
    return text

# Cleaning


In [87]:
df['tweet'] = df['tweet'].astype(str)

In [88]:
df['tweet_cleaned'] = df['tweet'].apply(cleaning)

# Casefolding

Mengubah huruf kapital menjadi huruf kecil

In [89]:
df['tweet_casefolded'] = df['tweet_cleaned'].str.lower()

# Drop Duplicate

In [90]:
duplicated_sum = df.duplicated('tweet_casefolded').sum() # Check Duplicated
duplicated_sum

4

In [91]:
duplicated_index = df.loc[df.duplicated('tweet_casefolded')].index
duplicated_index

Index([5106, 5562, 7082, 8184], dtype='int64')

In [92]:
print('number of data:',len(df))
print('duplicated data:',duplicated_sum)

df.drop_duplicates(subset= ['tweet_casefolded'],inplace=True)
df.reset_index(drop=True, inplace=True)

print('number of data after dropping duplicates:',len(df))

number of data: 9748
duplicated data: 4
number of data after dropping duplicates: 9744


# Tokenizing

In [93]:
df['tweet_tokenized'] = df['tweet_casefolded'].apply(word_tokenize)

# Normalisasi

https://github.com/riochr17/Analisis-Sentimen-ID/blob/master/preproses.py

In [94]:
data_kbbi

['anakjakartaasikasik\tanak jakarta asik asik',
 'pakcikdahtua\tpak cik sudah tua',
 'pakcikmudalagi\tpak cik muda lagi',
 't3tapjokowi\ttetap jokowi',
 '3x\ttiga kali',
 'aamiin\tamin',
 'aamiinn\tamin',
 'aamin\tamin',
 'aammiin\tamin',
 'abis\thabis',
 'abisin\thabiskan',
 'acau\tkacau',
 'achok\tahok',
 'ad\tada',
 'adek\tadik',
 'adl\tadalah',
 'adlah\tadalah',
 'adlh\tadalah',
 'ado\tada',
 'aduhh\taduh',
 'aer\tair',
 'afdol\tafdal',
 'agamaataualqur\tagama alquran',
 'agm\tagama',
 'agma\tagama',
 'ahaha\thaha',
 'ahahaha\thaha',
 'ahehehehe\thehe',
 'ahir\takhir',
 'ahirnya\takhirnya',
 'ahk\tahok',
 'ahlamdulillah\talhamdulillah',
 'ahli2\tpara ahli',
 'ahlusunnah\tahlus sunah',
 'ahmaddani\tahmad dhani',
 'aho\tahok',
 'ahoax\tahok',
 'ahoaxx\tahok',
 'ahog\tahok',
 'ahokataudjarot\tahok djarot',
 'ahokbebanijokowi\tahok beban jokowi',
 'ahokbtp\tahok basuki tjahaja purnama',
 'ahokditolakwarga\tahok ditolak warga',
 'ahokdjarot\tahok djarot',
 'ahokdjarotterbukti\tahok djar

In [95]:
#untuk memastikan setiap key memiliki value
tes_dic = {}
for i in data_kbbi:
    print(i)
    (key,val) = i.split('\t')
    tes_dic[str(key)] = val

anakjakartaasikasik	anak jakarta asik asik
pakcikdahtua	pak cik sudah tua
pakcikmudalagi	pak cik muda lagi
t3tapjokowi	tetap jokowi
3x	tiga kali
aamiin	amin
aamiinn	amin
aamin	amin
aammiin	amin
abis	habis
abisin	habiskan
acau	kacau
achok	ahok
ad	ada
adek	adik
adl	adalah
adlah	adalah
adlh	adalah
ado	ada
aduhh	aduh
aer	air
afdol	afdal
agamaataualqur	agama alquran
agm	agama
agma	agama
ahaha	haha
ahahaha	haha
ahehehehe	hehe
ahir	akhir
ahirnya	akhirnya
ahk	ahok
ahlamdulillah	alhamdulillah
ahli2	para ahli
ahlusunnah	ahlus sunah
ahmaddani	ahmad dhani
aho	ahok
ahoax	ahok
ahoaxx	ahok
ahog	ahok
ahokataudjarot	ahok djarot
ahokbebanijokowi	ahok beban jokowi
ahokbtp	ahok basuki tjahaja purnama
ahokditolakwarga	ahok ditolak warga
ahokdjarot	ahok djarot
ahokdjarotterbukti	ahok djarot terbukti
ahokers	ahoker
ahokfighter	ahok pejuang
ahokforpresident	ahok for presiden
ahokisthebest	ahok terbaik
ahokjarot	ahok djarot
ahokk	ahok
ahokmasukbui	ahok masuk bui
ahokmeskipun	ahok meskipun
ahokpositifjadi	ahok 

In [96]:
df['tweet_change_word'] = df['tweet_tokenized'].apply(change_word)

# Stopword removal

menghapus kata yang kurang penting, namun sering muncul (stopword)

In [97]:
df['tweet_removed_stop_word'] = df['tweet_change_word'].apply(stopwords_removal)

In [98]:
df

,index,Unnamed: 0,tweet,Labels,tweet_cleaned,tweet_casefolded,tweet_tokenized,tweet_change_word,tweet_removed_stop_word
0,0,0,@username sirait. sirait. kok otak kamu cetek ...,2,sirait sirait kok otak kamu cetek sangat ya...,sirait sirait kok otak kamu cetek sangat ya...,"[sirait, sirait, kok, otak, kamu, cetek, sanga...","[sirait, sirait, kok, otak, kamu, cetek, sanga...","[sirait, sirait, otak, cetek, iya, harga, miny..."
1,1,1,lain halnya islam yang mengelola sda minyak da...,1,lain halnya islam yang mengelola sda minyak da...,lain halnya islam yang mengelola sda minyak da...,"[lain, halnya, islam, yang, mengelola, sda, mi...","[lain, halnya, islam, yang, mengelola, sda, mi...","[halnya, islam, mengelola, minyak, gas, bumi, ..."
2,2,2,kenaikan bahan bakar minyak seringkali diikuti...,0,kenaikan bahan bakar minyak seringkali diikuti...,kenaikan bahan bakar minyak seringkali diikuti...,"[kenaikan, bahan, bakar, minyak, seringkali, d...","[kenaikan, bahan, bakar, minyak, seringkali, d...","[kenaikan, bahan, bakar, minyak, seringkali, d..."
3,3,3,"harga bahan bakar minyak naik, pemerintah dide...",0,harga bahan bakar minyak naik pemerintah dide...,harga bahan bakar minyak naik pemerintah dide...,"[harga, bahan, bakar, minyak, naik, pemerintah...","[harga, bahan, bakar, minyak, naik, pemerintah...","[harga, bahan, bakar, minyak, naik, pemerintah..."
4,4,4,"selesai zuhur angin bertiup kencang, entah apa...",0,selesai zuhur angin bertiup kencang entah apa...,selesai zuhur angin bertiup kencang entah apa...,"[selesai, zuhur, angin, bertiup, kencang, enta...","[selesai, zuhur, angin, bertiup, kencang, enta...","[selesai, zuhur, angin, bertiup, kencang, peny..."
...,...,...,...,...,...,...,...,...,...
9739,9746,9746,adanya lembaga penyalur tersebut akan kian mem...,1,adanya lembaga penyalur tersebut akan kian mem...,adanya lembaga penyalur tersebut akan kian mem...,"[adanya, lembaga, penyalur, tersebut, akan, ki...","[adanya, lembaga, penyalur, tersebut, akan, ki...","[lembaga, penyalur, mempermudah, masyarakat, m..."
9740,9747,9747,"harga bahan bakar minyak tidak naik, cuma peny...",2,harga bahan bakar minyak tidak naik cuma peny...,harga bahan bakar minyak tidak naik cuma peny...,"[harga, bahan, bakar, minyak, tidak, naik, cum...","[harga, bahan, bakar, minyak, tidak, naik, cum...","[harga, bahan, bakar, minyak, naik, penyesuaia..."
9741,9748,9748,sependapat dengan presiden jokowi bahwa sudah ...,1,sependapat dengan presiden jokowi bahwa sudah ...,sependapat dengan presiden jokowi bahwa sudah ...,"[sependapat, dengan, presiden, jokowi, bahwa, ...","[sependapat, dengan, presiden, jokowi, bahwa, ...","[sependapat, presiden, jokowi, bersyukur, peme..."
9742,9749,9749,ayo simak video di bawah ini video memperlihat...,0,ayo simak video di bawah ini video memperlihat...,ayo simak video di bawah ini video memperlihat...,"[ayo, simak, video, di, bawah, ini, video, mem...","[ayo, simak, video, di, bawah, ini, video, mem...","[ayo, simak, video, video, alasan, solusi, har..."


# Stemming

In [99]:
df['tweet_untokenized'] = df['tweet_removed_stop_word'].apply(untokenize)

In [100]:
df[['tweet_removed_stop_word','tweet_untokenized']].loc[0]

tweet_removed_stop_word    [sirait, sirait, otak, cetek, iya, harga, miny...
tweet_untokenized          sirait sirait otak cetek iya harga minyak turu...
Name: 0, dtype: object

In [101]:
df['tweet_stemmed'] = df['tweet_untokenized'].apply(stemming)

# Drop empty rows
Drop empty rows after preprocessing text data

In [102]:
empty_index = df['tweet_stemmed'].loc[df['tweet_stemmed'] == ''].index

In [103]:
print(len(df))
df['tweet_stemmed'].replace('',np.nan,inplace=True)
df.dropna(subset = ['tweet_stemmed'],inplace = True)
df.reset_index(drop=True,inplace=True)
print(len(df))

9744
9742


# Normalisasi

In [104]:
df[['tweet_tokenized','tweet_change_word']]

,tweet_tokenized,tweet_change_word
0,"[sirait, sirait, kok, otak, kamu, cetek, sanga...","[sirait, sirait, kok, otak, kamu, cetek, sanga..."
1,"[lain, halnya, islam, yang, mengelola, sda, mi...","[lain, halnya, islam, yang, mengelola, sda, mi..."
2,"[kenaikan, bahan, bakar, minyak, seringkali, d...","[kenaikan, bahan, bakar, minyak, seringkali, d..."
3,"[harga, bahan, bakar, minyak, naik, pemerintah...","[harga, bahan, bakar, minyak, naik, pemerintah..."
4,"[selesai, zuhur, angin, bertiup, kencang, enta...","[selesai, zuhur, angin, bertiup, kencang, enta..."
...,...,...
9737,"[adanya, lembaga, penyalur, tersebut, akan, ki...","[adanya, lembaga, penyalur, tersebut, akan, ki..."
9738,"[harga, bahan, bakar, minyak, tidak, naik, cum...","[harga, bahan, bakar, minyak, tidak, naik, cum..."
9739,"[sependapat, dengan, presiden, jokowi, bahwa, ...","[sependapat, dengan, presiden, jokowi, bahwa, ..."
9740,"[ayo, simak, video, di, bawah, ini, video, mem...","[ayo, simak, video, di, bawah, ini, video, mem..."


In [106]:
df['Labels'].value_counts()

Labels
2    3420
1    3203
0    3119
Name: count, dtype: int64

# Export data

In [107]:
df.to_csv('',index = False)